# Example-01: Optimizaion (import/export API)

In [1]:
# Loading API facilitates (suboptimal) interface with different optimization libraries
# In this example, quadrupole gradients are used to fit beta functions

In [2]:
from pathlib import Path

from model.external import load_sdds
from model.external import load_lattice
from model.external import text_lattice

import numpy
from numpy import ndarray as Array

from scipy.optimize import minimize

In [3]:
# Set quadrupole gradient and compute and return twiss parameters

def evaluate(knobs:Array) -> Array:
    kf, kd = knobs
    path:Path = Path('optimize.lte')
    lattice:dict[str, dict[str, str | int | float | dict]] = load_lattice(path)
    lattice['QF']['K1'] = float(kf)
    lattice['QD']['K1'] = float(kd)
    with path.open('w') as stream:
        stream.write(text_lattice('LTE', lattice))
    !elegant 'optimize.ele' > /dev/null
    !sddsconvert -ascii 'binary.twiss' 'optimize.twiss'
    path:Path = Path('optimize.twiss')
    _, columns = load_sdds(path)
    return numpy.asarray([[data['betax'], data['betay']] for location, data in columns.items()]).T

In [4]:
# Set target beta functions

target:Array = numpy.asarray([+0.21, -0.19])
result:Array = evaluate(target)

In [5]:
# Set objetive function to minimize

def objective(knobs:Array) -> Array:
    return numpy.sum((evaluate(knobs) - result)**2)

objective(target)

0.0

In [6]:
# Optimize 

knobs:Array = numpy.asarray([+0.20, -0.20])

minimize(objective, knobs, method='Nelder-Mead')

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 1.3489521479279302e-28
             x: [ 2.100e-01 -1.900e-01]
           nit: 20
          nfev: 40
 final_simplex: (array([[ 2.100e-01, -1.900e-01],
                       [ 2.100e-01, -1.900e-01],
                       [ 2.100e-01, -1.900e-01]]), array([ 1.349e-28,  2.891e-05,  6.010e-05]))

# Example-02: Workflow (MADX)

In [1]:
from pathlib import Path

from model.external import load_lattice
from model.external import rift_lattice
from model.external import text_lattice
from model.external import load_tfs
from model.external import convert
from model.external import add_rc

In [2]:
# Given some initial MADX lattice file (FODO)

file = Path('initial.madx')

with file.open('r') as stream:
    print(stream.read())

# Several regular elements are defined
# HEAD and TAIL should appear as the first and the last elements
# All elements should be defined on a single line with numerical parameters
# Lattice should be defined using lines
# Comma after element type is mandatory
# Comments appearing after definitions should also represent an element definition

DR: DRIFT, L=2.0;
BM: SBEND, L=1.0, ANGLE=0.17453292519943295;
QF: QUADRUPOLE, L=1.0, K1=+0.2;
QD: QUADRUPOLE, L=0.5, K1=-0.2;

M: MONITOR,;

HEAD: MARKER,; ! TEST: DRIFT,
TAIL: MARKER,; ! TEST: DRIFT,

FODO: LINE=(HEAD, M, QD, DR, BM, DR, QF, DR, BM, DR, QD, TAIL) ;



In [3]:
# If element and beamline definitions comply with the above requirements
# The lattice file can be loaded as a python dictionary

lattice = load_lattice(file)

for key, value in lattice.items():
    print(key, value)

# For each element and beamline, a key-value pair in created
# Value is itself a dictionary containing all information about the original elements
# Each element parameter is casted from string to int, float or string
# Comment after element definition is saved into RC (it has a special use case, see below)

DR {'KIND': 'DRIFT', 'RC': '', 'L': 2.0}
BM {'KIND': 'SBEND', 'RC': '', 'L': 1.0, 'ANGLE': 0.17453292519943295}
QF {'KIND': 'QUADRUPOLE', 'RC': '', 'L': 1.0, 'K1': 0.2}
QD {'KIND': 'QUADRUPOLE', 'RC': '', 'L': 0.5, 'K1': -0.2}
M {'KIND': 'MONITOR', 'RC': ''}
HEAD {'KIND': 'MARKER', 'RC': 'TEST: DRIFT,'}
TAIL {'KIND': 'MARKER', 'RC': 'TEST: DRIFT,'}
FODO {'KIND': 'LINE', 'SEQUENCE': ['HEAD', 'M', 'QD', 'DR', 'BM', 'DR', 'QF', 'DR', 'BM', 'DR', 'QD', 'TAIL']}


In [4]:
# Error lattice is defined by a set of linear transformations between selected locations
# Each locations can be a MONITOR (beam observation) or a VIRTUAL (error)
# Two special locatons (HEAD and TAIL) should present in the lattice
# Using the above dictionary representation, new observation locations can be inserted
# Locations are inserted at the middle of selected elements (selected by type or name)
# Selected elements are splitted in half and renamed, old names are binded to beamlines
# Original element definitions are added to created location RC
# Typicaly, monitor locations correspond to MONITOR elements, but new monitor elements can be also inserted
# Virtual locations can be inserted into quadrupole or other elements to represent errors

lattice = rift_lattice(lattice, 
                       'MONITOR', 
                       'MARKER', 
                       ['DRIFT'], 
                       ['SBEND', 'QUADRUPOLE'], 
                       exclude_virtual=['QD'])

for key, value in lattice.items():
    print(key, value)

M_DR {'KIND': 'MONITOR', 'RC': ['DR', {'KIND': 'DRIFT', 'L': 2.0}]}
H_DR {'KIND': 'DRIFT', 'L': 1.0}
DR {'KIND': 'LINE', 'SEQUENCE': ['H_DR', 'M_DR', 'H_DR']}
V_BM {'KIND': 'MARKER', 'RC': ['BM', {'KIND': 'SBEND', 'L': 1.0, 'ANGLE': 0.17453292519943295}]}
H_BM {'KIND': 'SBEND', 'L': 0.5, 'ANGLE': 0.08726646259971647}
BM {'KIND': 'LINE', 'SEQUENCE': ['H_BM', 'V_BM', 'H_BM']}
V_QF {'KIND': 'MARKER', 'RC': ['QF', {'KIND': 'QUADRUPOLE', 'L': 1.0, 'K1': 0.2}]}
H_QF {'KIND': 'QUADRUPOLE', 'L': 0.5, 'K1': 0.2}
QF {'KIND': 'LINE', 'SEQUENCE': ['H_QF', 'V_QF', 'H_QF']}
QD {'KIND': 'QUADRUPOLE', 'RC': '', 'L': 0.5, 'K1': -0.2}
M {'KIND': 'MONITOR', 'RC': ''}
HEAD {'KIND': 'MARKER', 'RC': 'TEST: DRIFT,'}
TAIL {'KIND': 'MARKER', 'RC': 'TEST: DRIFT,'}
FODO {'KIND': 'LINE', 'SEQUENCE': ['HEAD', 'M', 'QD', 'DR', 'BM', 'DR', 'QF', 'DR', 'BM', 'DR', 'QD', 'TAIL']}


In [5]:
# Modified lattice can be converted to text
# Comments are added to locations while original comments are preserved

text = text_lattice('MADX', lattice, rc=True)

print(text)

M_DR: MONITOR,; ! DR: DRIFT, L=2.0,;
H_DR: DRIFT, L=1.0,;
DR: LINE=(H_DR, M_DR, H_DR);
V_BM: MARKER,; ! BM: SBEND, L=1.0, ANGLE=0.17453292519943295,;
H_BM: SBEND, L=0.5, ANGLE=0.08726646259971647,;
BM: LINE=(H_BM, V_BM, H_BM);
V_QF: MARKER,; ! QF: QUADRUPOLE, L=1.0, K1=0.2,;
H_QF: QUADRUPOLE, L=0.5, K1=0.2,;
QF: LINE=(H_QF, V_QF, H_QF);
QD: QUADRUPOLE, L=0.5, K1=-0.2,;
M: MONITOR,;
HEAD: MARKER,; ! TEST: DRIFT,
TAIL: MARKER,; ! TEST: DRIFT,
FODO: LINE=(HEAD, M, QD, DR, BM, DR, QF, DR, BM, DR, QD, TAIL);



In [6]:
# Compute TWISS parameters using MADX
# TWISS command is appended to modified lattice

task = """
BEAM;
USE, PERIOD=FODO;
SET,FORMAT="20.20f","-20s";
TWISS;
WRITE,TABLE=TWISS,FILE="final.tfs";
RETURN;
""" ;

with Path('final.madx').open('w') as stream:
    stream.write(text)
    stream.write(task)

!madx final.madx > /dev/null

In [7]:
# Load lattice can be also loaded from file
# Original comments will be parsed as elements (look at HEAD and TAIL)
# Empty RC will be nested in this case

file = Path('final.madx')

with file.open('w') as stream:
    stream.write(text)

lattice = load_lattice(file, rc=True)

for key, value in lattice.items():
    print(key, value)

M_DR {'KIND': 'MONITOR', 'RC': ['DR', {'KIND': 'DRIFT', 'RC': '', 'L': 2.0}]}
H_DR {'KIND': 'DRIFT', 'RC': ['', {'KIND': '', 'RC': ''}], 'L': 1.0}
DR {'KIND': 'LINE', 'SEQUENCE': ['H_DR', 'M_DR', 'H_DR']}
V_BM {'KIND': 'MARKER', 'RC': ['BM', {'KIND': 'SBEND', 'RC': '', 'L': 1.0, 'ANGLE': 0.17453292519943295}]}
H_BM {'KIND': 'SBEND', 'RC': ['', {'KIND': '', 'RC': ''}], 'L': 0.5, 'ANGLE': 0.08726646259971647}
BM {'KIND': 'LINE', 'SEQUENCE': ['H_BM', 'V_BM', 'H_BM']}
V_QF {'KIND': 'MARKER', 'RC': ['QF', {'KIND': 'QUADRUPOLE', 'RC': '', 'L': 1.0, 'K1': 0.2}]}
H_QF {'KIND': 'QUADRUPOLE', 'RC': ['', {'KIND': '', 'RC': ''}], 'L': 0.5, 'K1': 0.2}
QF {'KIND': 'LINE', 'SEQUENCE': ['H_QF', 'V_QF', 'H_QF']}
QD {'KIND': 'QUADRUPOLE', 'RC': ['', {'KIND': '', 'RC': ''}], 'L': 0.5, 'K1': -0.2}
M {'KIND': 'MONITOR', 'RC': ['', {'KIND': '', 'RC': ''}]}
HEAD {'KIND': 'MARKER', 'RC': ['TEST', {'KIND': 'DRIFT', 'RC': ''}]}
TAIL {'KIND': 'MARKER', 'RC': ['TEST', {'KIND': 'DRIFT', 'RC': ''}]}
FODO {'KIND': '

In [8]:
# TWISS results can be loaded into python dictionaries

data = Path('final.tfs')
parameters, columns = load_tfs(data)

In [9]:
# Optics data can be converted into model table
# Note, all locations have different name
# If an element appear several times in a line, locations are renamed

table = convert(columns, 'TFS', ['MONITOR'], ['MARKER'], rc=True)
table

{'HEAD': {'TYPE': 'VIRTUAL',
  'S': 0.0,
  'BX': 4.287017735718936,
  'AX': -3.338e-16,
  'FX': 0.0,
  'BY': 19.818489282044894,
  'AY': -2.975e-17,
  'FY': 0.0,
  'DQX': 1.5490410441348796,
  'DPX': 5.551e-17,
  'DQY': 0.0,
  'DPY': -0.0,
  'RC': None},
 'M': {'TYPE': 'MONITOR',
  'S': 0.0,
  'BX': 4.287017735718936,
  'AX': -3.338e-16,
  'FX': 0.0,
  'BY': 19.818489282044894,
  'AY': -2.975e-17,
  'FY': 0.0,
  'DQX': 1.5490410441348796,
  'DPX': 5.551e-17,
  'DQY': 0.0,
  'DPY': -0.0,
  'RC': None},
 'M_DR': {'TYPE': 'MONITOR',
  'S': 1.5,
  'BX': 5.980356480296974,
  'AX': -0.8524040348462865,
  'FX': 0.3068185833848285,
  'BY': 15.315159900296642,
  'AY': 1.6491678474974667,
  'FY': 0.08453149247893033,
  'DQX': 1.744126900814982,
  'DPX': 0.1561982029636459,
  'DQY': 0.0,
  'DPY': 0.0,
  'RC': None},
 'V_BM': {'TYPE': 'VIRTUAL',
  'S': 3.0,
  'BX': 9.120629409698159,
  'AX': -1.1465687400023221,
  'FX': 0.5107301354647854,
  'BY': 10.914137614299745,
  'AY': 1.2848470098337954,
  

In [10]:
# RC parameter from lattice data can be added to model table
# Configuration table can be saved using util.save

table = add_rc(table, lattice)
table

{'HEAD': {'TYPE': 'VIRTUAL',
  'S': 0.0,
  'BX': 4.287017735718936,
  'AX': -3.338e-16,
  'FX': 0.0,
  'BY': 19.818489282044894,
  'AY': -2.975e-17,
  'FY': 0.0,
  'DQX': 1.5490410441348796,
  'DPX': 5.551e-17,
  'DQY': 0.0,
  'DPY': -0.0,
  'RC': None},
 'M': {'TYPE': 'MONITOR',
  'S': 0.0,
  'BX': 4.287017735718936,
  'AX': -3.338e-16,
  'FX': 0.0,
  'BY': 19.818489282044894,
  'AY': -2.975e-17,
  'FY': 0.0,
  'DQX': 1.5490410441348796,
  'DPX': 5.551e-17,
  'DQY': 0.0,
  'DPY': -0.0,
  'RC': None},
 'M_DR': {'TYPE': 'MONITOR',
  'S': 1.5,
  'BX': 5.980356480296974,
  'AX': -0.8524040348462865,
  'FX': 0.3068185833848285,
  'BY': 15.315159900296642,
  'AY': 1.6491678474974667,
  'FY': 0.08453149247893033,
  'DQX': 1.744126900814982,
  'DPX': 0.1561982029636459,
  'DQY': 0.0,
  'DPY': 0.0,
  'RC': ['DR', {'KIND': 'DRIFT', 'L': 2.0}]},
 'V_BM': {'TYPE': 'VIRTUAL',
  'S': 3.0,
  'BX': 9.120629409698159,
  'AX': -1.1465687400023221,
  'FX': 0.5107301354647854,
  'BY': 10.914137614299745,

# Example-03: Workflow (ELEGANT)

In [1]:
from pathlib import Path

from model.external import load_lattice
from model.external import rift_lattice
from model.external import text_lattice
from model.external import load_sdds
from model.external import convert
from model.external import add_rc

In [2]:
# Given some initial ELEGANT lattice file (FODO)

file = Path('initial.lte')

with file.open('r') as stream:
    print(stream.read())

# Several regular elements are defined
# HEAD and TAIL should appear as the first and the last elements
# All elements should be defined on a single line with numerical parameters
# Lattice should be defined using lines
# Comma after element type is mandatory
# Comments appearing after definitions should also represent an element definition

DR: DRIF,l=2

BM: CSBEND,angle=0.17453292519943295,fint=0,l=1.0
QD: QUAD,k1=-0.2,l=0.5
QF: QUAD,k1=0.2,l=1.0

M: MONI,

HEAD: MARK, ! TEST: DRIF,
TAIL: MARK, ! TEST: DRIF,

FODO: LINE=(HEAD, M, QD, DR, BM, DR, QF, DR, BM, DR, QD, TAIL)


In [3]:
# If element and beamline definitions comply with the above requirements
# The lattice file can be loaded as a python dictionary

lattice = load_lattice(file)

for key, value in lattice.items():
    print(key, value)

# For each element and beamline, a key-value pair in created
# Value is itself a dictionary containing all information about the original elements
# Each element parameter is casted from string to int, float or string
# Comment after element definition is saved into RC (it has a special use case, see below)

DR {'KIND': 'DRIF', 'RC': '', 'L': 2}
BM {'KIND': 'CSBEND', 'RC': '', 'ANGLE': 0.17453292519943295, 'FINT': 0, 'L': 1.0}
QD {'KIND': 'QUAD', 'RC': '', 'K1': -0.2, 'L': 0.5}
QF {'KIND': 'QUAD', 'RC': '', 'K1': 0.2, 'L': 1.0}
M {'KIND': 'MONI', 'RC': ''}
HEAD {'KIND': 'MARK', 'RC': 'TEST: DRIF,'}
TAIL {'KIND': 'MARK', 'RC': 'TEST: DRIF,'}
FODO {'KIND': 'LINE', 'SEQUENCE': ['HEAD', 'M', 'QD', 'DR', 'BM', 'DR', 'QF', 'DR', 'BM', 'DR', 'QD', 'TAIL']}


In [4]:
# Error lattice is defined by a set of linear transformations between selected locations
# Each locations can be a MONITOR (beam observation) or a VIRTUAL (error)
# Two special locatons (HEAD and TAIL) should present in the lattice
# Using the above dictionary representation, new observation locations can be inserted
# Locations are inserted at the middle of selected elements (selected by type or name)
# Selected elements are splitted in half and renamed, old names are binded to beamlines
# Original element definitions are added to created location RC
# Typicaly, monitor locations correspond to MONITOR elements, but new monitor elements can be also inserted
# Virtual locations can be inserted into quadrupole or other elements to represent errors

lattice = rift_lattice(lattice, 
                       'MONI', 
                       'MARK', 
                       ['DRIF'], 
                       ['CSBEND', 'QUAD'], 
                       exclude_virtual=['QD'])

for key, value in lattice.items():
    print(key, value)

M_DR {'KIND': 'MONI', 'RC': ['DR', {'KIND': 'DRIF', 'L': 2}]}
H_DR {'KIND': 'DRIF', 'L': 1.0}
DR {'KIND': 'LINE', 'SEQUENCE': ['H_DR', 'M_DR', 'H_DR']}
V_BM {'KIND': 'MARK', 'RC': ['BM', {'KIND': 'CSBEND', 'ANGLE': 0.17453292519943295, 'FINT': 0, 'L': 1.0}]}
H_BM {'KIND': 'CSBEND', 'ANGLE': 0.08726646259971647, 'FINT': 0, 'L': 0.5}
BM {'KIND': 'LINE', 'SEQUENCE': ['H_BM', 'V_BM', 'H_BM']}
QD {'KIND': 'QUAD', 'RC': '', 'K1': -0.2, 'L': 0.5}
V_QF {'KIND': 'MARK', 'RC': ['QF', {'KIND': 'QUAD', 'K1': 0.2, 'L': 1.0}]}
H_QF {'KIND': 'QUAD', 'K1': 0.2, 'L': 0.5}
QF {'KIND': 'LINE', 'SEQUENCE': ['H_QF', 'V_QF', 'H_QF']}
M {'KIND': 'MONI', 'RC': ''}
HEAD {'KIND': 'MARK', 'RC': 'TEST: DRIF,'}
TAIL {'KIND': 'MARK', 'RC': 'TEST: DRIF,'}
FODO {'KIND': 'LINE', 'SEQUENCE': ['HEAD', 'M', 'QD', 'DR', 'BM', 'DR', 'QF', 'DR', 'BM', 'DR', 'QD', 'TAIL']}


In [5]:
# Modified lattice can be converted to text
# Comments are added to locations while original comments are preserved

text = text_lattice('LTE', lattice, rc=True)

print(text)

M_DR: MONI, ! DR: DRIF, L=2,
H_DR: DRIF, L=1.0,
DR: LINE=(H_DR, M_DR, H_DR)
V_BM: MARK, ! BM: CSBEND, ANGLE=0.17453292519943295, FINT=0, L=1.0,
H_BM: CSBEND, ANGLE=0.08726646259971647, FINT=0, L=0.5,
BM: LINE=(H_BM, V_BM, H_BM)
QD: QUAD, K1=-0.2, L=0.5,
V_QF: MARK, ! QF: QUAD, K1=0.2, L=1.0,
H_QF: QUAD, K1=0.2, L=0.5,
QF: LINE=(H_QF, V_QF, H_QF)
M: MONI,
HEAD: MARK, ! TEST: DRIF,
TAIL: MARK, ! TEST: DRIF,
FODO: LINE=(HEAD, M, QD, DR, BM, DR, QF, DR, BM, DR, QD, TAIL)



In [6]:
# Compute TWISS parameters using ELEGANT
# Separate command file is created

with Path('final.lte').open('w') as stream:
    stream.write(text)

task = """
&run_setup
  use_beamline="FODO",
  lattice = "final.lte",
  p_central_mev = 1000
&end

&run_control
&end

&twiss_output
  filename = "binary.twiss",
  output_at_each_step = 1
&end

&bunched_beam
&end

&track
&end
""" ;

with Path('final.ele').open('w') as stream:
    stream.write(task)

!elegant final.ele > /dev/null
!sddsconvert -ascii binary.twiss final.twiss

In [7]:
# Load lattice can be also loaded from file
# Original comments will be parsed as elements (look at HEAD and TAIL)
# Empty RC will be nested in this case

file = Path('final.lte')

with file.open('w') as stream:
    stream.write(text)

lattice = load_lattice(file, rc=True)

for key, value in lattice.items():
    print(key, value)

M_DR {'KIND': 'MONI', 'RC': ['DR', {'KIND': 'DRIF', 'RC': '', 'L': 2}]}
H_DR {'KIND': 'DRIF', 'RC': ['', {'KIND': '', 'RC': ''}], 'L': 1.0}
DR {'KIND': 'LINE', 'SEQUENCE': ['H_DR', 'M_DR', 'H_DR']}
V_BM {'KIND': 'MARK', 'RC': ['BM', {'KIND': 'CSBEND', 'RC': '', 'ANGLE': 0.17453292519943295, 'FINT': 0, 'L': 1.0}]}
H_BM {'KIND': 'CSBEND', 'RC': ['', {'KIND': '', 'RC': ''}], 'ANGLE': 0.08726646259971647, 'FINT': 0, 'L': 0.5}
BM {'KIND': 'LINE', 'SEQUENCE': ['H_BM', 'V_BM', 'H_BM']}
QD {'KIND': 'QUAD', 'RC': ['', {'KIND': '', 'RC': ''}], 'K1': -0.2, 'L': 0.5}
V_QF {'KIND': 'MARK', 'RC': ['QF', {'KIND': 'QUAD', 'RC': '', 'K1': 0.2, 'L': 1.0}]}
H_QF {'KIND': 'QUAD', 'RC': ['', {'KIND': '', 'RC': ''}], 'K1': 0.2, 'L': 0.5}
QF {'KIND': 'LINE', 'SEQUENCE': ['H_QF', 'V_QF', 'H_QF']}
M {'KIND': 'MONI', 'RC': ['', {'KIND': '', 'RC': ''}]}
HEAD {'KIND': 'MARK', 'RC': ['TEST', {'KIND': 'DRIF', 'RC': ''}]}
TAIL {'KIND': 'MARK', 'RC': ['TEST', {'KIND': 'DRIF', 'RC': ''}]}
FODO {'KIND': 'LINE', 'SEQUEN

In [8]:
# TWISS results can be loaded into python dictionaries

data = Path('final.twiss')
parameters, columns = load_sdds(data)

In [9]:
# Optics data can be converted into model table
# Note, all locations have different name
# If an element appear several times in a line, locations are renamed

table = convert(columns, 'SDDS', ['MONI'], ['MARK'], rc=True)
table

{'HEAD': {'S': 0.0,
  'BX': 4.287017734831204,
  'AX': -1.321304551729011e-16,
  'FX': 0.0,
  'DQX': 1.549040841795901,
  'DPX': 2.775557561562891e-17,
  'BY': 19.81848926815186,
  'AY': 6.545730027929358e-16,
  'FY': 0.0,
  'DQY': 0.0,
  'DPY': 0.0,
  'TYPE': 'VIRTUAL',
  'RC': None},
 'M': {'S': 0.0,
  'BX': 4.287017734831204,
  'AX': -1.321304551729011e-16,
  'FX': 0.0,
  'DQX': 1.549040841795901,
  'DPX': 2.775557561562891e-17,
  'BY': 19.81848926815186,
  'AY': 6.545730027929358e-16,
  'FY': 0.0,
  'DQY': 0.0,
  'DPY': 0.0,
  'TYPE': 'MONITOR',
  'RC': None},
 'M_DR': {'S': 1.5,
  'BX': 5.980356479284525,
  'AX': -0.8524040348212205,
  'FX': 0.3068185834444499,
  'DQX': 1.744126672993481,
  'DPX': 0.1561981825607101,
  'BY': 15.31515988971357,
  'AY': 1.649167846239909,
  'FY': 0.08453149253790615,
  'DQY': 0.0,
  'DPY': 0.0,
  'TYPE': 'MONITOR',
  'RC': None},
 'V_BM': {'S': 3.0,
  'BX': 9.120629409314306,
  'AX': -1.146568740048648,
  'FX': 0.5107301355818847,
  'DQX': 1.9928963

In [10]:
# RC parameter from lattice data can be added to model table
# Configuration table can be saved using util.save

table = add_rc(table, lattice)
table

{'HEAD': {'S': 0.0,
  'BX': 4.287017734831204,
  'AX': -1.321304551729011e-16,
  'FX': 0.0,
  'DQX': 1.549040841795901,
  'DPX': 2.775557561562891e-17,
  'BY': 19.81848926815186,
  'AY': 6.545730027929358e-16,
  'FY': 0.0,
  'DQY': 0.0,
  'DPY': 0.0,
  'TYPE': 'VIRTUAL',
  'RC': None},
 'M': {'S': 0.0,
  'BX': 4.287017734831204,
  'AX': -1.321304551729011e-16,
  'FX': 0.0,
  'DQX': 1.549040841795901,
  'DPX': 2.775557561562891e-17,
  'BY': 19.81848926815186,
  'AY': 6.545730027929358e-16,
  'FY': 0.0,
  'DQY': 0.0,
  'DPY': 0.0,
  'TYPE': 'MONITOR',
  'RC': None},
 'M_DR': {'S': 1.5,
  'BX': 5.980356479284525,
  'AX': -0.8524040348212205,
  'FX': 0.3068185834444499,
  'DQX': 1.744126672993481,
  'DPX': 0.1561981825607101,
  'BY': 15.31515988971357,
  'AY': 1.649167846239909,
  'FY': 0.08453149253790615,
  'DQY': 0.0,
  'DPY': 0.0,
  'TYPE': 'MONITOR',
  'RC': ['DR', {'KIND': 'DRIF', 'L': 2}]},
 'V_BM': {'S': 3.0,
  'BX': 9.120629409314306,
  'AX': -1.146568740048648,
  'FX': 0.51073013